In [1]:
import pandas as pd
import numpy as np
import os
import pydicom
import matplotlib
import matplotlib.pyplot as plt 
import cv2
from tqdm import tqdm

In [2]:
CBIS_dir = "/datasets/mammography/CBIS-DDSM/"

In [60]:
mass_train = pd.read_csv("/datasets/mammography/mass_case_description_train_set.csv")
mass_test = pd.read_csv("/datasets/mammography/mass_case_description_test_set.csv")
calc_train = pd.read_csv("/datasets/mammography/calc_case_description_train_set.csv")
calc_test = pd.read_csv("/datasets/mammography/calc_case_description_test_set.csv")

In [143]:
mass_train

,patient_id,breast_density,left or right breast,image view,abnormality id,abnormality type,mass shape,mass margins,assessment,pathology,subtlety,image file path,cropped image file path,ROI mask file path
0,P_00001,3,LEFT,CC,1,mass,IRREGULAR-ARCHITECTURAL_DISTORTION,SPICULATED,4,MALIGNANT,4,Mass-Training_P_00001_LEFT_CC/1.3.6.1.4.1.9590...,Mass-Training_P_00001_LEFT_CC_1/1.3.6.1.4.1.95...,Mass-Training_P_00001_LEFT_CC_1/1.3.6.1.4.1.95...
1,P_00001,3,LEFT,MLO,1,mass,IRREGULAR-ARCHITECTURAL_DISTORTION,SPICULATED,4,MALIGNANT,4,Mass-Training_P_00001_LEFT_MLO/1.3.6.1.4.1.959...,Mass-Training_P_00001_LEFT_MLO_1/1.3.6.1.4.1.9...,Mass-Training_P_00001_LEFT_MLO_1/1.3.6.1.4.1.9...
2,P_00004,3,LEFT,CC,1,mass,ARCHITECTURAL_DISTORTION,ILL_DEFINED,4,BENIGN,3,Mass-Training_P_00004_LEFT_CC/1.3.6.1.4.1.9590...,Mass-Training_P_00004_LEFT_CC_1/1.3.6.1.4.1.95...,Mass-Training_P_00004_LEFT_CC_1/1.3.6.1.4.1.95...
3,P_00004,3,LEFT,MLO,1,mass,ARCHITECTURAL_DISTORTION,ILL_DEFINED,4,BENIGN,3,Mass-Training_P_00004_LEFT_MLO/1.3.6.1.4.1.959...,Mass-Training_P_00004_LEFT_MLO_1/1.3.6.1.4.1.9...,Mass-Training_P_00004_LEFT_MLO_1/1.3.6.1.4.1.9...
4,P_00004,3,RIGHT,MLO,1,mass,OVAL,CIRCUMSCRIBED,4,BENIGN,5,Mass-Training_P_00004_RIGHT_MLO/1.3.6.1.4.1.95...,Mass-Training_P_00004_RIGHT_MLO_1/1.3.6.1.4.1....,Mass-Training_P_00004_RIGHT_MLO_1/1.3.6.1.4.1....
5,P_00009,3,RIGHT,CC,1,mass,OVAL,ILL_DEFINED,4,MALIGNANT,4,Mass-Training_P_00009_RIGHT_CC/1.3.6.1.4.1.959...,Mass-Training_P_00009_RIGHT_CC_1/1.3.6.1.4.1.9...,Mass-Training_P_00009_RIGHT_CC_1/1.3.6.1.4.1.9...
6,P_00009,3,RIGHT,MLO,1,mass,OVAL,ILL_DEFINED,4,MALIGNANT,4,Mass-Training_P_00009_RIGHT_MLO/1.3.6.1.4.1.95...,Mass-Training_P_00009_RIGHT_MLO_1/1.3.6.1.4.1....,Mass-Training_P_00009_RIGHT_MLO_1/1.3.6.1.4.1....
7,P_00015,3,LEFT,MLO,1,mass,IRREGULAR,ILL_DEFINED-SPICULATED,4,MALIGNANT,4,Mass-Training_P_00015_LEFT_MLO/1.3.6.1.4.1.959...,Mass-Training_P_00015_LEFT_MLO_1/1.3.6.1.4.1.9...,Mass-Training_P_00015_LEFT_MLO_1/1.3.6.1.4.1.9...
8,P_00018,2,RIGHT,CC,1,mass,OVAL,CIRCUMSCRIBED,4,BENIGN,5,Mass-Training_P_00018_RIGHT_CC/1.3.6.1.4.1.959...,Mass-Training_P_00018_RIGHT_CC_1/1.3.6.1.4.1.9...,Mass-Training_P_00018_RIGHT_CC_1/1.3.6.1.4.1.9...
9,P_00018,2,RIGHT,MLO,1,mass,OVAL,CIRCUMSCRIBED,4,BENIGN,5,Mass-Training_P_00018_RIGHT_MLO/1.3.6.1.4.1.95...,Mass-Training_P_00018_RIGHT_MLO_1/1.3.6.1.4.1....,Mass-Training_P_00018_RIGHT_MLO_1/1.3.6.1.4.1....


In [113]:
calc_train.iloc[33]["ROI mask file path"]

'Calc-Training_P_00020_LEFT_CC_1/1.3.6.1.4.1.9590.100.1.2.380637862511101891719875529443997027392/1.3.6.1.4.1.9590.100.1.2.270637349710602065435194182703174870832/000000.dcm'

In [129]:
file_path_list = {"PatientFolder": [], "Path": [], "Type": []}
for path, subdirs, files in os.walk(CBIS_dir):
    for name in files:
        entry_dict = dict()
        entry = os.path.join(path, name).split("/")[4:]
        file_path_list["PatientFolder"].append(entry[0])
        file_path_list["Path"].append(os.path.join(*entry))
        if "ROI" in entry[2] and name == "000000.dcm":
            file_path_list["Type"].append("ROI mask file path")
        elif ("ROI" in entry[2] and name == "000001.dcm") or "cropped" in entry[2] and name == "000000.dcm":
            file_path_list["Type"].append("cropped image file path")
        else:
            file_path_list["Type"].append("image file path")
my_df = pd.DataFrame(file_path_list, columns=["PatientFolder", "Path", "Type"])

In [130]:
my_df.head(20)

,PatientFolder,Path,Type
0,Calc-Training_P_00868_RIGHT_MLO_1,Calc-Training_P_00868_RIGHT_MLO_1/09-06-2017-D...,cropped image file path
1,Calc-Training_P_00868_RIGHT_MLO_1,Calc-Training_P_00868_RIGHT_MLO_1/09-06-2017-D...,ROI mask file path
2,Calc-Training_P_01319_RIGHT_MLO_1,Calc-Training_P_01319_RIGHT_MLO_1/09-06-2017-D...,cropped image file path
3,Calc-Training_P_01319_RIGHT_MLO_1,Calc-Training_P_01319_RIGHT_MLO_1/09-06-2017-D...,ROI mask file path
4,Calc-Test_P_00620_LEFT_CC,Calc-Test_P_00620_LEFT_CC/08-29-2017-DDSM-4441...,image file path
5,Calc-Training_P_01436_RIGHT_CC_1,Calc-Training_P_01436_RIGHT_CC_1/09-06-2017-DD...,cropped image file path
6,Calc-Training_P_01436_RIGHT_CC_1,Calc-Training_P_01436_RIGHT_CC_1/09-06-2017-DD...,ROI mask file path
7,Calc-Training_P_00635_RIGHT_MLO,Calc-Training_P_00635_RIGHT_MLO/08-07-2016-DDS...,image file path
8,Calc-Test_P_00077_RIGHT_CC,Calc-Test_P_00077_RIGHT_CC/08-29-2017-DDSM-381...,image file path
9,Calc-Training_P_00624_LEFT_MLO_1,Calc-Training_P_00624_LEFT_MLO_1/09-06-2017-DD...,cropped image file path


In [148]:
image_file_path = []
cropped_image_path = []
roi_mask_path = []
for index, row in mass_test.iterrows():
    patient_folder = row['image file path'].split('/')[0]
    image_file_path.append(my_df[my_df['PatientFolder'] == patient_folder].iloc[0]["Path"])
    cropped_folder = row["cropped image file path"].split('/')[0]
    for _, row_1 in my_df[my_df["PatientFolder"] == cropped_folder].iterrows():
        if row_1['Type'] == "ROI mask file path":
            roi_mask_path.append(row_1['Path'])
        elif row_1["Type"] == "cropped image file path":
            cropped_image_path.append(row_1['Path'])
        else:
            roi_mask_path.append(row_1['Path'])

In [149]:
len(image_file_path), len(cropped_image_path), len(roi_mask_path)

(378, 378, 378)

In [146]:
mass_test["image file path"] = image_file_path
mass_test["cropped image file path"] = cropped_image_path
mass_test["ROI mask file path"] = image_file_path

In [150]:
mass_test.to_csv("/datasets/mammography/new_mass_case_description_test_set.csv")

In [7]:
for root, subFolder, file_names in os.walk(CBIS_dir):
    for file_name in file_names:
        dicom_path = os.path.join(root, file_name)
        out_path = os.path.join(root, file_name)
        out_path = out_path.split(".dcm")[0] + ".png"
        image = pydicom.dcmread(dicom_path).pixel_array
        image = cv2.resize(image, (1152, 832), interpolation = cv2.INTER_AREA)
        status = cv2.imwrite(out_path,  cv2.normalize(image, None, alpha = 0, 
                                      beta = 255, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_32F))

TypeError: 'module' object is not callable